In [42]:
### 0. import
import psycopg2
import numpy  as np
import pandas as pd
from datetime import datetime


### 1. 데이터 가져오기
# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

In [43]:
# 상가 면적 (pnu단위_19자리)
sql = '''
select main_purps_cd_nm, tot_area, sigungu_cd||bjdong_cd||case when plat_gb_cd = '1' then '2' else '1' end||bun||ji as pnu
  from m1.bld_rgst
 where main_purps_cd_nm not in 
    (
    select distinct main_purps_cd_nm
    from m1.bld_rgst
    where main_purps_cd_nm like '%주택%'
    )
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_bld_rgstd = pd.DataFrame(rows)
df_bld_rgstd.columns = ['main_purps_cd_nm','tot_area','pnu']

In [44]:
df_bld_rgstd

,main_purps_cd_nm,tot_area,pnu
0,문화및집회시설,187.72,1111010100100030100
1,제1종근린생활시설,744.82,1111010100100040020
2,제1종근린생활시설,43.32,1111010100100070001
3,제1종근린생활시설,81.90,1111010100100070003
4,제1종근린생활시설,388.96,1111010100100150005
...,...,...,...
2976542,동.식물관련시설,99.00,5013032026119660000
2976543,동.식물관련시설,66.00,5013032026119710000
2976544,창고시설,72.00,5013032026120160000
2976545,창고시설,100.80,5013032026120270000


In [45]:
# 거주 인구 (법정동 단위)
sql = 'select emd_cd, l_tot, base_ym from m2.cremao_local_population'
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_cremao_local_population = pd.DataFrame(rows)
df_cremao_local_population.columns = ['emd_cd','l_tot','base_ym']

In [46]:
df_cremao_local_population

,emd_cd,l_tot,base_ym
0,41480350,210,202208
1,41480360,87,202208
2,41480370,70,202208
3,41480380,8,202208
4,41480390,0,202208
...,...,...,...
5049,47130140,72,202208
5050,47130141,28,202208
5051,47130142,88,202208
5052,47130143,60,202208


In [36]:
df_cremao_local_population['tot_area_sum'] = None

In [32]:
# 법정동 매칭
idx = df_bld_rgstd[(df_bld_rgstd['pnu'].str[:8]) == df_cremao_local_population['emd_cd'][i]].index

In [34]:
# sum
df_cremao_local_population['tot_area_sum'][i] = df_bld_rgstd['tot_area'][idx].sum()

1715887.6466

In [ ]:
# 인구 대비 연면적
df_cremao_local_population['tot_area_sum'] / df_cremao_local_population['l_tot']

In [ ]:
# 법정동 매칭 (한 pnu안에 여러 개의 건물이 있으므로 sum)
if (df_bld_rgstd['pnu'].str[:19]) == df_cremao_local_population['emd_cd']:
    df_bld_rgstd[(df_bld_rgstd['pnu'].str[:19]) == df_cremao_local_population['emd_cd']] # sum
    df_bld_rgstd['tot_area'][i] / df_cremao_local_population['l_tot'][j] # 나누기

In [ ]:
df_cremao_local_population['tot_area_sum'] = None

for i in df_cremao_local_population.index:
    
    idx = df_bld_rgstd[(df_bld_rgstd['pnu'].str[:8]) == df_cremao_local_population['emd_cd'][i]].index # 법정동 매칭
    df_cremao_local_population['tot_area_sum'][i] = df_bld_rgstd['tot_area'][idx].sum() # sum
    df_cremao_local_population['tot_area_sum'] / df_cremao_local_population['l_tot'] # 인구 대비 연면적

In [ ]:
### 3. DB INSERT
import datetime

table = 'm1.'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))